<a href="https://colab.research.google.com/github/SAMAWELJABALLI/TweetsSentimentAnalysis/blob/main/procecessing_of_processed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import resample
    
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Load data and set labels from corpus 1 
data = pd.read_csv('/content/data_processed.csv')
print(f'Shape of data is {data.shape}')
# Display 20 random samples
data.sample(20)

Shape of data is (199097, 3)


,ID,text_clean,label
19313,19313,شد دارك,1
180830,51038,bravo drmarzouki et je dis bien errajel rajel,1
90017,43407,jai aimé toutes mes commandes ont été suivies ...,1
97913,51962,très bien comme application surtout avec le co...,1
17333,17333,برافو سيدي الوزير,1
56614,7253,je que là bonne expérience,1
167559,37274,tahya lclub wtabga sloma,1
96547,50469,بيتم استبدال المنتجات بمنتجات غير اصليه لا انص...,0
66405,17881,حلو جدااااااااا,1
149859,18947,nn miskin houwa chy iiasefff fina walah w en t...,0


# Duplicate Sentence

Most webscrapped data contains duplicated sentence.This function could retrieve unique ones.

In [2]:
def pick_unique_sentence(text):
    line=re.findall(r'(?sm)(^[^\r\n]+$)(?!.*^\1$)', text)
    return line


In [3]:
sentence="ya wa7ch dima a3ml talla\n I thank doctors\nDoctors are working very hard in this pandemic situation\nI thank doctors bravo lik y wa7ch \nya wa7ch dima a3ml talla\n  "
pick_unique_sentence(sentence)

[' I thank doctors',
 'Doctors are working very hard in this pandemic situation',
 'I thank doctors bravo lik y wa7ch ',
 'ya wa7ch dima a3ml talla',
 '  ']

In [4]:
data["text_clean"].fillna("No content", inplace = True)
data['pick_unique']=data['text_clean'].apply(lambda x : pick_unique_sentence(x))
data.sample(20)

,ID,text_clean,label,pick_unique
40642,40642,يا توفيق رك مصت برش,0,[يا توفيق رك مصت برش]
51690,1950,le meilleur des site en vente en ligne,1,[le meilleur des site en vente en ligne]
89644,43014,site arnaqueur de lux ils se foutent de tous l...,0,[site arnaqueur de lux ils se foutent de tous ...
127673,22928,donnez l exemple et mettez une bavette svp,1,[donnez l exemple et mettez une bavette svp]
27082,27082,No content,1,[No content]
13082,13082,ربي يقدر الخير,1,[ربي يقدر الخير]
28915,28915,No content,1,[No content]
54005,4454,امىتتبتتى,1,[امىتتبتتى]
23658,23658,ربنا يرفع عنا وعنكو البلاء,1,[ربنا يرفع عنا وعنكو البلاء]
50765,953,null null nulllll,0,[null null nulllll]


# remove repeated lettre

In [5]:
def rep(text):
    grp = text.group(0)
    if len(grp) > 1:
        return grp[0:1] # can change the value here on repetition
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert


In [6]:
sentence="مااااسط مااااسط bravoooooooooooooo degaage"
unique_char(rep,sentence)

'ماسط ماسط bravo degage'

In [7]:
data['pick_unique']=data['pick_unique'].apply(str)
data['unique_char']=data['pick_unique'].apply(lambda x : unique_char(rep,x))
data.head(20)

,ID,text_clean,label,pick_unique,unique_char
0,0,weldin weldikk bravooooo,1,['weldin weldikk bravooooo'],['weldin weldik bravo']
1,1,bravo amine j aime,1,['bravo amine j aime'],['bravo amine j aime']
2,2,صراحة bravo أحمد الرحموني,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني']
3,3,souut w ziiin lee wlh bch f,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f']
4,4,anaa,1,['anaa'],['ana']
5,5,bravooooo walid,1,['bravooooo walid'],['bravo walid']
6,6,محترم و نموة علها,1,['محترم و نموة علها'],['محترم و نموة علها']
7,7,ربي يشفيلك بوك وان شاء الله غصرة وتتعدى,1,['ربي يشفيلك بوك وان شاء الله غصرة وتتعدى'],['ربي يشفيلك بوك وان شاء اله غصرة وتعدى']
8,8,bravoooooooooooooooo nabara jnab rak,1,['bravoooooooooooooooo nabara jnab rak'],['bravo nabara jnab rak']
9,9,bravo,1,['bravo'],['bravo']


# 1-grame features extracted from processed_text

In [8]:
def ngrams_top(corpus,ngram_range,n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(ngram_range=ngram_range).fit(corpus.values.astype('U'))
    bag_of_words = vec.transform((corpus).values.astype('U'))
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    return df

In [ ]:
une_gram =ngrams_top(data['unique_char'],(1,1),n=200)
une_gram.values.tolist()


In [52]:
noise1gram=set(['wiw','inti','inty','na','je','wé','ena','cete','رجلرجل','eni','ana','انت','ben','عن','al','mes','و','w','mel','wel','jour','come','هو','بعد','inti','الى','se','lkol','fama','bel','فيه','ele','فى','kima','haka','kan','nes','بينا','votre','par','ya','mr','une', 'cest','vous','ont','sont','nous','kif','علي','faire','dans','fel','ala','ana','taw','ye','mon','tout','bravobravo','col','مع','li','ça','de','ya','في','et','fi','el','je','من','les','pour','est','على','il','des','si','vous','en','que','cest','الي','kol','eli','un','يا','une','ما','bech','ou','ken','كل','men','كان','jai','باش','ama','mala','cette','هذا','انا','ان','avec','du','dima','ce','min','ili','ki','enti','fih','tawa','qui','au','ena','fil','suis','wa'])
len(noise1gram)

112

# 2-grame features extracted from processed_text


In [ ]:
deux_gram =ngrams_top(data['unique_char'],(2,2),n=100)
deux_gram.values.tolist()


# 3-grame features extracted from processed_text


In [ ]:
trois_gram =ngrams_top(data['unique_char'],(3,3),n=100)
trois_gram.values.tolist()


# 4-grame features extracted from processed_text


In [ ]:
quatre_gram =ngrams_top(data['unique_char'],(4,4),n=100)
quatre_gram.values.tolist()


# length of words
No regex but added one liner to identify length of words in a sentence


In [13]:
data['text_length']=data['unique_char'].str.split().map(lambda x: len(x))
data[['unique_char','text_length']].sample(20)


,unique_char,text_length
41096,['عاهرة عاهرة عاهرة عاهرة عاهرة عاهرة عاهرة عا...,33
69437,['çest un trés bon service merci davoir crier'],8
67406,['لو سمحت انا بدور على موبيل برو هو متوفر عندك...,25
70884,['est ce que la livraison est gratuite sinon t...,12
117586,['كارثة واله ومازال الشي مادام العروسات لوجه ا...,8
117401,['واله لعباد مازالت شايخة والسوق دزني وندزك وا...,10
178719,['wow crazy wish'],3
181259,['eneyeka radithom mriglin enti mrigl ye stana...,17
4742,['وجيهة تعجبني وتشرف التمثيل والمسرح في تونس ر...,9
74888,['younes hala'],2


number of words on cleaned corpus

In [14]:
data['text_length'].sum()

1583826

# Length of characters
No regex but added one liner to identify length of words in a sentence



In [15]:
data['char_length']=data['unique_char'].str.len()
data[['unique_char','char_length']].sample(3)

,unique_char,char_length
189698,['alohoma ethrib ethalimin b thalimin w minhom...,68
80346,['excelent aplication bravo'],29
143028,['la inahikom'],15


number of characters on cleaned corpus

In [16]:
data['char_length'].sum()

9333187

In [17]:
data.drop(['text_clean', 'text_length','char_length'], axis=1, inplace=True)

In [47]:
data['text_cleaned'] = data['unique_char'].apply(lambda s: " ".join([word for word in s.split() if word not in noise1gram]))
data.head(5)

,ID,label,pick_unique,unique_char,text_cleaned
0,0,1,['weldin weldikk bravooooo'],['weldin weldik bravo'],['weldin weldik bravo']
1,1,1,['bravo amine j aime'],['bravo amine j aime'],['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني']
3,3,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f'],['sout zin le wlh bch f']
4,4,1,['anaa'],['ana'],['ana']


# 2,3,4-grmaes after removing inutile words from 1-grmaes

In [ ]:
deux_gram =ngrams_top(data['text_cleaned'],(2,2),n=200)
deux_gram.values.tolist()

In [ ]:
trois_gram =ngrams_top(data['text_cleaned'],(3,3),n=200)
trois_gram.values.tolist()

In [ ]:
quatre_gram =ngrams_top(data['text_cleaned'],(4,4),n=200)
quatre_gram.values.tolist()

In [ ]:
n_gram =ngrams_top(data['text_cleaned'],(8,8),n=200)
n_gram.values.tolist()

In [54]:
data['text_length']=data['text_cleaned'].str.split().map(lambda x: len(x))
data[['text_cleaned','text_length']].sample(20)
data['text_length'].sum()

1320253

In [55]:
1583826-1320253

263573

c'est deja pas mal de libérer 263573 mots

In [73]:
def unifier(phrase):
    # specific
    phrase = re.sub(r"trés", "tres", phrase)
    phrase = re.sub(r"très", "tres", phrase)
    phrase = re.sub(r"apli","application",phrase)
    phrase = re.sub(r"lapli","application",phrase)
    phrase = re.sub(r"l'aplication","application",phrase)
    phrase = re.sub(r"l'application","application",phrase)
    phrase = re.sub(r"aplication","application",phrase)
    
    return phrase

data['text_cleaned']=data['text_cleaned'].apply(lambda s: unifier(s))

In [74]:
data

,ID,label,pick_unique,unique_char,text_cleaned,text_length
0,0,1,['weldin weldikk bravooooo'],['weldin weldik bravo'],['weldin weldik bravo'],3
1,1,1,['bravo amine j aime'],['bravo amine j aime'],['bravo amine j aime'],4
2,2,1,['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],['صراحة bravo أحمد الرحموني'],4
3,3,1,['souut w ziiin lee wlh bch f'],['sout w zin le wlh bch f'],['sout zin le wlh bch f'],6
4,4,1,['anaa'],['ana'],['ana'],1
...,...,...,...,...,...,...
199092,69995,1,['amazing love'],['amazing love'],['amazing love'],2
199093,69996,1,['ena raniiiiiiiiiii eriahi ca bolomètre kleb ...,['ena rani eriahi ca bolomètre kleb leyeme bin...,['ena rani eriahi ca bolomètre kleb leyeme bin...,15
199094,69997,0,['hada maycharafech tounis'],['hada maycharafech tounis'],['hada maycharafech tounis'],3
199095,69998,0,['leken rajlek haw wemcher inteviou'],['leken rajlek haw wemcher inteviou'],['leken rajlek haw wemcher inteviou'],5


In [75]:
data.drop(['pick_unique', 'unique_char','text_length'], axis=1, inplace=True)

In [77]:
data.head()

,ID,label,text_cleaned
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']


In [121]:
#save our clean dataframe into csv file
#data.to_csv ('cleaned_data.csv')
data.rename(index=str, columns={'text_cleaned':'text_clean'},inplace=True)
#save our clean dataframe into csv file
data.head()

,ID,label,text_clean
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']


In [122]:
data.to_csv('cleaned_data.csv',index=False)

In [123]:
data = pd.read_csv('/content/cleaned_data.csv')
data.head()

,ID,label,text_clean
0,0,1,['weldin weldik bravo']
1,1,1,['bravo amine j aime']
2,2,1,['صراحة bravo أحمد الرحموني']
3,3,1,['sout zin le wlh bch f']
4,4,1,['ana']
